## This notebook is the modified version of the notebook available here:
https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/question_answering.ipynb

## Run this notebook with a GPU enabled Runtime

## Following packages need to be installed

In [ ]:
!pip install datasets
!pip install transformers
!pip install torch

^C


You should consider upgrading via the 'c:\users\arnab\documents\python\bert_training\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## bquad The json format is similar to that of squad's.

For reference see the explore_bquad.ipynb notebook.

In [3]:
import datasets
import transformers
import torch
transformers.__version__, datasets.__version__, torch.__version__

('4.10.2', '1.11.0', '1.9.0+cu102')

In [2]:
bquad_path = '/content/drive/MyDrive/Colab Notebooks/data/bquad.json'
bquad_script_path = '/content/drive/MyDrive/Colab Notebooks/data/load_bquad.py'
modelpath = '/content/drive/MyDrive/Colab Notebooks/model/newmodel/'

In [3]:
import json

with open(bquad_path, 'r', encoding='UTF-8') as f:
    bquad = json.load(f)

data = bquad['data']
print(len(data), 'titles')
paras, qas = 0, 0

for i in range(len(data)):
    paras+= len(data[i]['paragraphs'])
    for j in range(len(data[i]['paragraphs'])):
        qas+= len(data[i]['paragraphs'][j]['qas'])
        # for k in range(len(data[i]['paragraphs'][j]['qas'])):
        #     value = data[i]['paragraphs'][j]['qas'][k]['is_impossible']
        #     if isinstance(value, str):
        #         data[i]['paragraphs'][j]['qas'][k]['is_impossible'] = False

print(paras, 'paragraphs', qas, 'qa pairs')

bquad['data'] = data

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/data/bquad.json'

## load_bquad.py is a custom script to load data from the bquad json file. 
## In the load_bquad.py change the URL variable to the directory where bquad is stored.

In [6]:
squad_v2 = False
from datasets import load_dataset, load_metric

datasets = load_dataset(bquad_script_path)

print(datasets)

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/ef1e39b1e98389c254fade8294da9fcc7e2fe18019d8e3dbf5956c1fad872263. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 13048
    })
})


## Random Exmaples

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

print(show_random_elements(datasets["train"]))

,id,title,context,question,answers
0,bquad--fgce-1011170,সোমালিয়া,"প্রাচীন সোমালিয়ায় সোমালিতে টালো নামে পরিচিত পিরামিডিকাল কাঠামো একটি জনপ্রিয় কবর রচনা ছিল, আজ শত শত শুকনো পাথরের স্মৃতিস্তম্ভগুলি আজ সারা দেশে ছড়িয়ে ছিটিয়ে রয়েছে। প্রাচীন মিশরে সদৃশ সদৃশ পোষাক দ্বারা ঘরগুলি নির্মিত হয়েছিল। আঙ্গিনা এবং বড় পাথরের দেয়াল বেষ্টনী বেষ্টন উদাহরণ, যেমন ওয়ারগাড ওয়াল।",সোমালীরা তাদের প্রাচীন পিরামিডগুলি কী বলে?,"{'text': ['টালো'], 'answer_start': [29]}"
1,bquad--fgce-1006265,পুনর্নবীকরণযোগ্য_সংক্রান্ত_ বাণিজ্যিকীকরণ,"জলবিদ্যুৎ শব্দটি জলবিদ্যুৎ দ্বারা উত্পাদিত বিদ্যুতকে বোঝায়; ঝর্ণা বা প্রবাহিত জল মহাকর্ষ শক্তি ব্যবহারের মাধ্যমে বৈদ্যুতিক শক্তি উত্পাদন। এটি পুনর্নবীকরণযোগ্য শক্তির সর্বাধিক ব্যবহৃত ফর্ম, এটি বিশ্ব বিদ্যুৎ উৎপাদনের ১ শতাংশ - ২০১০ সালে ৩,৪২27 টেরোয়াট-ঘন্টা বিদ্যুৎ উত্পাদন, এবং আগামী ২৫ বছরের জন্য প্রতি বছর প্রায় ৩.১% বাড়বে বলে আশা করা হচ্ছে। জলবিদ্যুৎ কেন্দ্রগুলি দীর্ঘজীবী হওয়ার সুবিধা রয়েছে এবং প্রচুর বিদ্যমান উদ্ভিদ 100 বছরেরও বেশি সময় ধরে পরিচালনা করে আসছে।",জলবিদ্যুৎ দ্বারা উত্পাদিত বিদ্যুতের কথা বলতে কী বোঝায়?,"{'text': ['জলবিদ্যুৎ'], 'answer_start': [0]}"
2,bquad--fgce-1007574,লাল,"প্রাচীন রোমে, টাইরিয়ান বেগুনি সম্রাটের রঙ ছিল, তবে লাল একটি গুরুত্বপূর্ণ ধর্মীয় প্রতীক ছিল। রোমানরা ছুটির দিনে লাল ফিতেযুক্ত টোগাস পরত, এবং একটি বিবাহের কনে একটি লাল শাল পরত, जिसे ফ্লেমিয়াম বলা হত। লাল মূর্তি এবং গ্ল্যাডিয়েটারদের ত্বক রঙ করতে ব্যবহৃত হত। লালও ছিল সেনাবাহিনীর সাথে সম্পর্কিত রঙ; রোমান সৈন্যরা লাল টুনিক পড়ত, এবং আধিকারিকরা একটি পলুডামেন্টাম নামে একটি পোশাক ছিল যা রঙের মানের উপর নির্ভর করে ক্রিমসন, স্কারলেট বা বেগুনি হতে পারে। রোমান পুরাণে লাল যুদ্ধের দেবতা, মঙ্গল গ্রহের সাথে জড়িত। রোমান সাম্রাজ্যের ভেক্সিলয়েডের স্বর্ণের এসপিকিউআর অক্ষরগুলির একটি লাল পটভূমি ছিল। একজন রোমান জেনারেল বিজয়ী হয়েছিলেন তার কৃতিত্বের সম্মানে তার পুরো শরীরটি লাল রঙ করেছিলেন।",কোন সময় লাল রঙের টোগাসগুলি রোমানরা পরত?,"{'text': ['ছুটি'], 'answer_start': [102]}"
3,bquad--fgce-1012188,ব্রক লেসনার,"ব্রক এডোয়ার্ড লেসনার [1] (/ˈlɛznər/;জন্ম ১২ জুলাই ১৯৭৭ )[1] একজন আমেরিকান পেশাদার কুস্তিগীর এবং পূর্বে অপেশাদার কুস্তিগীর, পেশাদার আমেরিকান ফুটবল খেলোয়াড়, এবং মিশ্র মার্শাল শিল্পী,[2] বর্তমানে সে ডাব্লিউডাব্লিউই এর ডাব্লিউডাব্লিউই র ব্রান্ডের সাথে চুক্তিবদ্ধ আছেন।[3] লেসনার ৪ বার ডাব্লিউডাব্লিউই (ওয়ার্ল্ড হেভি অয়েট ) চ্যাম্পিয়ন হয়েছেন,পূর্বে ইউ এফ সি ওয়ার্ল্ড হেভিওয়েট ছিলেন , এবং এনসিএএ বিভাগে কলেজিয়েট হেভিওয়েট চ্যাম্পিয়ান হন। ,[4][5]",পেশাদার কুস্তিগীর ব্রক লেসনার কবে জন্মগ্রহণ করেন ?,"{'text': ['১২ জুলাই ১৯৭৭'], 'answer_start': [43]}"
4,bquad--fgce-1002862,বিমান_বাহক,"আধুনিক নৌবাহিনী যারা এ জাতীয় বিমানবাহী বাহক পরিচালনা করে তাদের বহরের মূলধন জাহাজ হিসাবে বিবেচনা করে, যা আগে যুদ্ধজাহাজের ভূমিকা ছিল। এই পরিবর্তনটি দ্বিতীয় বিশ্বযুদ্ধের সময় বায়ু শক্তি যুদ্ধের একটি গুরুত্বপূর্ণ কারণ হয়ে ওঠে, উচ্চতর পরিসর দ্বারা চালিত, ক্যারিয়ার-চালু বিমানের নমনীয়তা এবং কার্যকারিতা দ্বারা পরিচালিত হওয়ার প্রতিক্রিয়াতে ঘটেছিল। যুদ্ধের পরে, ক্যারিয়ার অপারেশন আকার এবং গুরুত্ব বৃদ্ধি অবিরত। ৫,০০০ টন বা তার বেশি সংখ্যক স্থানচ্যুত সুপার কেরিয়ারগুলি ক্যারিয়ার বিকাশের শিখরে পরিণত হয়েছে। কিছু কিছু পারমাণবিক চুল্লি দ্বারা চালিত হয় এবং বাড়ি থেকে দূরে পরিচালনার জন্য নির্মিত একটি বহরের মূল গঠন করে। ইউএসএস তারাওয়া এবং এইচএমএস মহাসাগরের মতো উভচর হামলাকারী জাহাজগুলি মেরিনকে বহন এবং অবতরণের লক্ষ্যে কাজ করে এবং এই উদ্দেশ্যে হেলিকপ্টারগুলির একটি বিশাল দল পরিচালনা করে। ""কমান্ডো ক্যারিয়ার"" বা ""হেলিকপ্টার ক্যারিয়ার"" নামে পরিচিত, অনেকের ভিএসটিএল বিমান চালনা করার ক্ষমতা রয়েছে।",কোন জাহাজ মেরিন বহন এবং অবতরণের জন্য ব্যবহৃত হয়?,"{'text': ['উভচর হামলাকারী জাহাজ'], 'answer_start': [663]}"
5,bquad--fgce-1010958,জিন,"ডিএনএর দুটি শৃঙ্খলা একে অপরের চারপাশে মোচড় করে ডিএসএ ডাবল হেলিক্স গঠন করে ফসফেট-চিনি ব্যাকবোনটি বাইরের চারপাশে ঘূর্ণায়মান হয় এবং বেসগুলি থাইমাইন এবং গুয়ানিনকে সাইটোসিনের সাথে জোড়া দিয়ে অ্যাডিনাইন বেসের সাথে ভিতরের দিকে ইশারা করে। বেস পেয়ারিংয়ের বৈশিষ্ট্য দেখা দেয় কারণ অ্যাডেনিন এবং থাইমাইন দুটি হাইড্রোজেন বন্ধন গঠন করে, যেখানে সাইটোসিন এবং গুয়াইন তিনটি হাইড্রোজেন বন্ধন গঠন করে। ডাবল হেলিক্সের দুটি স্ট্র্যান্ড অবশ্যই পরিপূরক হতে হ

None


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(modelpath)
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

print(tokenizer("আজ অবধি, বেশিরভাগ শিকারী সংগ্রহকারী শ্রমের প্রতীকী কাঠামোগত যৌন বিভাগ রয়েছে"))
print(tokenizer.decode(tokenizer("আজ অবধি, বেশিরভাগ শিকারী সংগ্রহকারী শ্রমের প্রতীকী কাঠামোগত যৌন বিভাগ রয়েছে")['input_ids']))

for i, example in enumerate(datasets["train"]):
    # print(example["question"])
    # print(len(tokenizer(example["question"])["input_ids"]))
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]
# print(datasets["train"][i])

print(len(tokenizer(example["question"], example["context"])["input_ids"]))

{'input_ids': [2, 1932, 12277, 16, 6602, 33137, 48293, 24889, 18247, 32560, 3743, 2253, 2015, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] আজ অবধি, বেশিরভাগ শিকারী সংগরহকারী শরমের পরতীকী কাঠামোগত যৌন বিভাগ রযেছে [SEP]
387


In [9]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

print([len(x) for x in tokenized_example["input_ids"]])

for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[384, 147]
[CLS] গলযাজড কালেকশন সিসটেমের চেযে কি পরিবহন সংগরহকারীরা কম বেশি সাশরযী? [SEP] সৌর কেনদরীকরণ পরযকতি যেমন পযারাবোলিক ডিশ, টরাট এবং শেফলার পরতিবিমবগলি বাণিজযিক ও শিলপকৌশল অযাপলিকেশনগলির জনয পরকরিযা তাপ সরবরাহ করতে পারে । পরথম বাণিজযিক বযবসথাটি মারকিন যকতরাষটরের জরজিযার শেনানডোহায সোলার টোটাল এনারজি পরজেকট ( এসটিইপি ) ছিল যেখানে ১১৪ টি পযারাবলিক খাবারের কষেতরটি পোশাক কারখানার জনয পরকরিযাজাতীয হিটিং, শীতাতপ নিযনতরণ এবং বৈদযতিক পরযোজনীযতার 50 % সরবরাহ করে । এই গরিড - সংযকত সমনবয বযবসথা 401 কিলোওযাট বাষপ এবং 468 কিলোওযাট শীতল জল আকারে 400 কিলোওযাট বিদযত পলাস তাপীয শকতি সরবরাহ করেছিল এবং এতে এক ঘনটা শিখর লোড তাপীয সটোরেজ ছিল । বাষপীভবন পকরগলি অগভীর পল যা বাষপীভবনের মাধযমে দরবীভত ঘনকে ঘনীভত করে । সমদরের জল থেকে লবণ গরহণের জনয বাষপীভবনের পকরগলির বযবহার সৌরশকতির অনযতম পরাচীনতম পরযোগ । আধনিক বযবহারগলির মধযে জলাবদধতা খনির কষেতরে বযবহত বরিন সলিউশনগলিকে কেনদরীভত করা এবং বরজয পরবাহগলি থেকে দরবীভত দরবণগলি অপসারণ অনতরভকত । কাপডের লাইনগলি, কাপডের ঘোডাগলি এবং কাপডগলি বিদযৎ বা গ

In [10]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[(0, 0), (0, 6), (6, 8), (9, 16), (17, 26), (27, 32), (33, 35), (36, 42), (43, 53), (53, 55), (56, 58), (59, 63), (64, 72), (72, 73), (0, 0), (0, 3), (4, 13), (13, 15), (16, 25), (26, 30), (31, 37), (37, 40), (40, 42), (43, 46), (46, 47), (48, 52), (52, 53), (54, 57), (58, 61), (61, 64), (65, 75), (75, 79), (80, 89), (90, 91), (92, 99), (98, 100), (100, 101), (102, 114), (114, 119), (120, 124), (125, 135), (136, 139), (140, 146), (147, 151), (152, 156), (156, 157), (158, 163), (164, 173), (174, 184), (185, 192), (193, 207), (208, 217), (218, 221), (221, 224), (224, 227), (227, 228), (230, 235), (236, 241), (242, 249), (250, 258), (259, 260), (260, 264), (264, 265), (265, 267), (267, 268), (269, 272), (273, 279), (280, 283), (284, 286), (287, 293), (293, 297), (298, 305), (306, 315), (316, 321), (322, 330), (331, 335), (336, 349), (349, 351), (353, 356), (356, 358), (358, 359), (360, 366), (367, 377), (378, 381), (382, 391), (392, 406), (407, 409), (409, 410), (411, 417), (418, 421), (4

In [11]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")


pad_on_right = tokenizer.padding_side == "right"

306 306


In [12]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [13]:
# features = prepare_train_features(datasets['train'][:5])
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/14 [00:00<?, ?ba/s]

In [14]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 13115
    })
})


In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

batch_size = 8

model = AutoModelForQuestionAnswering.from_pretrained(modelpath)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/model/newmodel/ were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering 

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [16]:
args = TrainingArguments(
    f"test-squad",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    do_eval = False,
    # evaluation_strategy= 'steps',
    # eval_steps = 1500,

)

# TrainingArguments()
# Trainer()

from transformers import default_data_collator

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

***** Running training *****
  Num examples = 13115
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6560


Step,Training Loss
500,3.190000
1000,2.401600
1500,2.087000
2000,1.712500
2500,1.559500
3000,1.567600
3500,1.353000
4000,1.212100
4500,1.176500
5000,1.149900


Saving model checkpoint to test-squad/checkpoint-500
Configuration saved in test-squad/checkpoint-500/config.json
Model weights saved in test-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-squad/checkpoint-1000
Configuration saved in test-squad/checkpoint-1000/config.json
Model weights saved in test-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-squad/checkpoint-1500
Configuration saved in test-squad/checkpoint-1500/config.json
Model weights saved in test-squad/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-15

TrainOutput(global_step=6560, training_loss=1.556086097112516, metrics={'train_runtime': 2893.2554, 'train_samples_per_second': 18.132, 'train_steps_per_second': 2.267, 'total_flos': 3048422538516480.0, 'train_loss': 1.556086097112516, 'epoch': 4.0})

In [17]:
savepath = './qamodel/'
trainer.save_model(savepath)

Saving model checkpoint to ./qamodel/
Configuration saved in ./qamodel/config.json
Model weights saved in ./qamodel/pytorch_model.bin
tokenizer config file saved in ./qamodel/tokenizer_config.json
Special tokens file saved in ./qamodel/special_tokens_map.json


## Test the model's performance on unseen data

In [18]:
import numpy as np
import torch
from transformers import BertForQuestionAnswering, AutoTokenizer
import transformers

model = BertForQuestionAnswering.from_pretrained(savepath)

tokenizer = AutoTokenizer.from_pretrained(savepath)   
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

loading configuration file ./qamodel/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/model/newmodel/",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 75000
}

loading weights file ./qamodel/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the model checkpoin

In [19]:
''''বাংলাদেশ সেনাবাহিনীকে ভারত ১২০ মিলিমিটার ব্যাসার্ধের ১৮টি মর্টার দিয়েছে। এটা দুই দেশের সেনাবাহিনীর মধ্যে দ্বিপক্ষীয় সহযোগিতার অংশ বলে জানিয়েছে ভারত। 
প্রধানমন্ত্রী শেখ হাসিনার সঙ্গে ভারতের প্রধানমন্ত্রী নরেন্দ্র মোদির ভার্চ্যুয়াল বৈঠকের পর বাংলাদেশ ও মিয়ানমারের ভারপ্রাপ্ত ভারতীয় পররাষ্ট্র মন্ত্রণালয়ের যুগ্ম সচিব স্মিতা পন্থ আজ বৃহস্পতিবার সন্ধ্যায় সংবাদমাধ্যমকে 
এই তথ্য জানান। গতকাল বুধবার বাংলাদেশের বিজয় দিবস উদ্‌যাপন উপলক্ষে মর্টারগুলো বাংলাদেশকে দিয়েছে ভারত।'
'''


questions = ['বাংলাদেশ সেনাবাহিনীকে কয়টি মর্টার দিয়েছে?', 
             'বাংলাদেশ সেনাবাহিনীকে ভারত কি দিয়েছে?', 
             'বাংলাদেশ সেনাবাহিনীকে মর্টার কে দিয়েছে?',  
             'দুই দেশের সেনাবাহিনীর মধ্যে দ্বিপক্ষীয় সহযোগিতার অংশ কোনটি?',
             'প্রধানমন্ত্রী শেখ হাসিনার সঙ্গে কার ভার্চ্যুয়াল বৈঠক হয়?',
             'মিয়ানমারের ভারপ্রাপ্ত ভারতীয় পররাষ্ট্র মন্ত্রণালয়ের যুগ্ম সচিব কে',
             'স্মিতা পন্থ কে?', 
             'মর্টারগুলো বাংলাদেশকে কেন দিয়েছে ভারত?',
             'মর্টারগুলো বাংলাদেশকে কি উপলক্ষে দিয়েছে ভারত?',
             ]       

question = 'মর্টারগুলো বাংলাদেশকে কি উপলক্ষে দিয়েছে ভারত?'
paragraph = 'বাংলাদেশ সেনাবাহিনীকে ভারত ১২০ মিলিমিটার ব্যাসার্ধের ১৮টি মর্টার দিয়েছে। এটা দুই দেশের সেনাবাহিনীর মধ্যে দ্বিপক্ষীয় সহযোগিতার অংশ বলে জানিয়েছে ভারত। প্রধানমন্ত্রী শেখ হাসিনার সঙ্গে ভারতের প্রধানমন্ত্রী নরেন্দ্র মোদির ভার্চ্যুয়াল বৈঠকের পর বাংলাদেশ ও মিয়ানমারের ভারপ্রাপ্ত ভারতীয় পররাষ্ট্র মন্ত্রণালয়ের যুগ্ম সচিব স্মিতা পন্থ আজ বৃহস্পতিবার সন্ধ্যায় সংবাদমাধ্যমকে এই তথ্য জানান। গতকাল বুধবার বাংলাদেশের বিজয় দিবস উদ্‌যাপন উপলক্ষে মর্টারগুলো বাংলাদেশকে দিয়েছে ভারত।'

for question in questions:
    input = tokenizer(question, paragraph, add_special_tokens=True, return_tensors="pt")
    temp = input["input_ids"].tolist()
    input_ids = temp[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # print(inputs)
    # tokenizer.decode(input_ids)
    outputs = model(**input)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # print(start_scores, end_scores)

    start_index = torch.argmax(start_scores)

    end_index = torch.argmax(end_scores)

    answer = ' '.join(text_tokens[start_index:end_index+1])
    corrected_answer = ''

    for word in answer.split():

        #If it's a subword token
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word

    corrected_answer = corrected_answer
    print('Ques :', question)
    print('Ans: ', corrected_answer)

Ques : বাংলাদেশ সেনাবাহিনীকে কয়টি মর্টার দিয়েছে?
Ans:   ১২০ মিলিমিটার বযাসারধের ১৮টি
Ques : বাংলাদেশ সেনাবাহিনীকে ভারত কি দিয়েছে?
Ans:   মরটার
Ques : বাংলাদেশ সেনাবাহিনীকে মর্টার কে দিয়েছে?
Ans:   ভারত
Ques : দুই দেশের সেনাবাহিনীর মধ্যে দ্বিপক্ষীয় সহযোগিতার অংশ কোনটি?
Ans:   মরটার
Ques : প্রধানমন্ত্রী শেখ হাসিনার সঙ্গে কার ভার্চ্যুয়াল বৈঠক হয়?
Ans:   ভারতের পরধানমনতরী নরেনদর মোদির
Ques : মিয়ানমারের ভারপ্রাপ্ত ভারতীয় পররাষ্ট্র মন্ত্রণালয়ের যুগ্ম সচিব কে
Ans:   সমিতা পনথ
Ques : স্মিতা পন্থ কে?
Ans:   বাংলাদেশ ও মিযানমারের ভারপরাপত ভারতীয পররাষটর মনতরণালযের যগম সচিব
Ques : মর্টারগুলো বাংলাদেশকে কেন দিয়েছে ভারত?
Ans:   ১২০ মিলিমিটার বযাসারধের ১৮টি মরটার দিযেছে । এটা দই দেশের সেনাবাহিনীর মধযে দবিপকষীয সহযোগিতার অংশ বলে জানিযেছে ভারত । পরধানমনতরী শেখ হাসিনার সঙগে ভারতের পরধানমনতরী নরেনদর মোদির ভারচযযাল বৈঠকের পর বাংলাদেশ ও মিযানমারের ভারপরাপত ভারতীয পররাষটর মনতরণালযের যগম সচিব সমিতা পনথ আজ বহসপতিবার সনধযায সংবাদমাধযমকে এই তথয জানান । গতকাল বধবার বাংলাদেশের বিজয দিবস উদযাপন উপলকষে
Ques : ম

# Validation

In [20]:
import torch

for batch in trainer.get_train_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
    
output.keys(), output.start_logits.shape, output.end_logits.shape

(odict_keys(['loss', 'start_logits', 'end_logits']),
 torch.Size([8, 384]),
 torch.Size([8, 384]))

In [21]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 52,  21,  47,  59,  90, 162,  11,  89], device='cuda:0'),
 tensor([ 54,  22,  51,  60,  91, 163,  11,  92], device='cuda:0'))

In [22]:
import numpy as np

n_best_size = 20

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [23]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [24]:
validation_features = datasets["train"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["train"].column_names
)

  0%|          | 0/14 [00:00<?, ?ba/s]

In [25]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 13115
  Batch size = 8


In [26]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

max_answer_length = 30

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["train"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[]

In [27]:
import collections

examples = datasets["train"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [28]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [29]:
final_predictions = postprocess_qa_predictions(datasets["train"], validation_features, raw_predictions.predictions)

Post-processing 13048 example predictions split into 13115 features.


  0%|          | 0/13048 [00:00<?, ?it/s]

In [32]:
metric = load_metric('squad')

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

In [33]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["train"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 56.53740036787247, 'f1': 72.43036578481133}